In [ ]:
# 기본코드
import inspect
import os
import sys
import time
import numpy as np

import torch
from mmcv import Config
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_model
from mmseg.apis import single_gpu_test
from mmcv.runner import load_checkpoint, load_state_dict
from mmcv.parallel import MMDataParallel

def uniform_soup(cfg, model, checkpoint_paths ,device = "cpu", by_name = False):
    dataset = build_dataset(cfg.data.val)
    data_loader = build_dataloader(
            dataset,
            samples_per_gpu=1,
            workers_per_gpu=cfg.data.workers_per_gpu,
            dist=False,
            shuffle=False)  
    
    model = model.to(device)
    model_dict = model.state_dict()
    soups = {key:[] for key in model_dict}
    checkpoint = {}
    for i, checkpoint_path in enumerate(checkpoint_paths):
        checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')
        weight_dict = checkpoint['state_dict']
        for k, v in weight_dict.items():
            soups[k].append(v)
    if 0 < len(soups):
        soups = {k:(torch.sum(torch.stack(v), axis = 0) / len(v)).type(v[0].dtype) for k, v in soups.items() if len(v) != 0}
        model_dict.update(soups)
        model.load_state_dict(model_dict)
    
    load_state_dict(model, model_dict)
    model.CLASSES = dataset.CLASSES
    model = MMDataParallel(model.cuda(), device_ids=[0])
    output = single_gpu_test(model, data_loader)
    eval_kwargs = {}
    eval_kwargs.update(metric=['mIoU'])
    metric = dataset.evaluate(output, **eval_kwargs)
    print(f"mIoU: {metric['mIoU']}")
    
    return model, checkpoint

def greedy_soup(cfg, model_ori, checkpoint_paths, device = "cpu"):
    dataset = build_dataset(cfg.data.val)
    data_loader = build_dataloader(
            dataset,
            samples_per_gpu=1,
            workers_per_gpu=cfg.data.workers_per_gpu,
            dist=False,
            shuffle=False)  
    
    result = []
    checkpoint = {}
    for i, checkpoint_path in enumerate(checkpoint_paths):
        model = model_ori.to(device)
        checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')
        model.CLASSES = dataset.CLASSES
        model = MMDataParallel(model.cuda(), device_ids=[0])
        output = single_gpu_test(model, data_loader)
        eval_kwargs = {}
        eval_kwargs.update(metric=['mIoU'])
        metric = dataset.evaluate(output, **eval_kwargs)
        result.append((metric['mIoU'],checkpoint_path))
        print(f"리스트에 {i}번째 mIoU {metric['mIoU']}저장")
    
    result.sort(key = lambda x : x[0], reverse = True)
    print(f"리스트 정렬")
    print(result)
    
    model = model_ori.to(device)
    model_dict = model.state_dict()
    pre_metric_value = 0
    pre_weight_dict = {}
    for i, (mIoU, checkpoint_path) in enumerate(result):
        model = model_ori.to(device)
        soups = {key:[] for key in model_dict}
        now_model_dict = model_dict
        if i == 0: 3 4 1 2
            checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')
            pre_metric_value = mIoU
            pre_weight_dict = checkpoint['state_dict']
            print("soup 모델에 가장 높은 mIou를 가진 checkpoint가 추가되었습니다")
            print(f"추가된 checkpoint_path: {checkpoint_path}")
            print(f"현재 최고 mIoU: {pre_metric_value}")
        else:
            checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')
            weight_dict = checkpoint['state_dict']
            
            for k, v in pre_weight_dict.items():
                soups[k].append(v)
            for k, v in weight_dict.items():
                soups[k].append(v)    
            if 0 < len(soups):
                soups = {k:(torch.sum(torch.stack(v), axis = 0) / len(v)).type(v[0].dtype) for k, v in soups.items() if len(v) != 0}
                now_model_dict.update(soups)
                
                
            load_state_dict(model, now_model_dict)
            model.CLASSES = dataset.CLASSES
            model = MMDataParallel(model.cuda(), device_ids=[0])
            output = single_gpu_test(model, data_loader)
            eval_kwargs = {}
            eval_kwargs.update(metric=['mIoU'])
            metric = dataset.evaluate(output, **eval_kwargs)
            
            if metric['mIoU'] >= pre_metric_value:
                pre_metric_value = metric['mIoU']
                pre_weight_dict = now_model_dict
                print("soup 모델에 새로운 checkpoint가 추가되었습니다")
                print(f"추가된 checkpoint_path: {checkpoint_path}")
                print(f"현재 최고 mIoU: {pre_metric_value}")
            else:
                print("이번 체크 포인트는 soup 모델에 추가되지 않았습니다")
                print(f"이번 checkpoint_path: {checkpoint_path}")
                print(f"현재 최고 mIoU: {pre_metric_value}, 이번 mIou {metric['mIoU']}")
            
    model = model_ori.to(device)
    load_state_dict(model, pre_weight_dict)
    return model, checkpoint

In [ ]:
################ model cfg path 적기 ################
cfg= Config.fromfile('/opt/ml/input/mmseg/work_dirs/knet_s3_upernet_swin-l_8x2_512x512_adamw_80k_ade20k/knet_s3_upernet_swin-l_8x2_512x512_adamw_80k_ade20k.py')
################ model cfg path 적기 ################
model = build_segmentor(cfg.model)

################ soup할 checkpoint path 적기 ################
checkpoint_paths = [
    '/opt/ml/input/mmseg/work_dirs/knet_s3_upernet_swin-l_8x2_512x512_adamw_80k_ade20k/epoch_18.pth',
    '/opt/ml/input/mmseg/work_dirs/knet_s3_upernet_swin-l_8x2_512x512_adamw_80k_ade20k/2_epoch_30.pth',
    '/opt/ml/input/mmseg/work_dirs/knet_s3_upernet_swin-l_8x2_512x512_adamw_80k_ade20k/1_best_mIoU_epoch_12.pth',
    '/opt/ml/input/mmseg/work_dirs/knet_s3_upernet_swin-l_8x2_512x512_adamw_80k_ade20k/epoch_20.pth',
]
################ soup할 checkpoint path 적기 ################
device = "cpu"

In [ ]:
################ save dir path 적기 ################
save_dir_path = '/opt/ml/input/mmseg/work_dirs/knet_s3_upernet_swin-l_8x2_512x512_adamw_80k_ade20k/'
name = 'mysoup' # soup 이름 적기
################ save dir path 적기 ################

print("\n[Uniform Soup]")
uniform_model, checkpoint = uniform_soup(cfg, model, checkpoint_paths, device = device)
uniform_dict = checkpoint
uniform_dict['state_dict'] = uniform_model.state_dict()

torch.save(uniform_dict, save_dir_path+f'uniform_model_soup_{name}.pth')

In [ ]:
################ save dir path 적기 ################
save_dir_path = '/opt/ml/input/mmseg/work_dirs/knet_s3_upernet_swin-l_8x2_512x512_adamw_80k_ade20k/'
name = 'mysoup' # soup 이름 적기
################ save dir path 적기 ################

print("[Greedy Soup (uniform weight update)]")
greedy_model, checkpoint = greedy_soup(cfg, model, checkpoint_paths, device = device)
greedy_dict = checkpoint
greedy_dict['state_dict'] = greedy_model.state_dict()
torch.save(greedy_dict, save_dir_path+f'greedy_model_soup_{name}.pth')

# Uniform Soup

In [5]:
# 기본코드
import os
import sys
import time
import numpy as np
import torch
from mmengine.runner import load_checkpoint
from mmseg.apis import init_model, inference_model

def uniform_soup(cfg, checkpoint_paths, device = "cuda", by_name = False):
    checkpoint = {}
    print(checkpoint_paths)        
    
    model = init_model(config_path, checkpoint_paths[0], device)
    model = model.to(device)
    model_dict = model.state_dict()
    soups = {key:[] for key in model_dict}
    for i, checkpoint_path in enumerate(checkpoint_paths):
        model = init_model(config_path, checkpoint_path, device)
        model = model.to(device)
        model_dict = model.state_dict()
        for k, v in model_dict.items():
            soups[k].append(v)
    if 0 < len(soups):
        soups = {k:(torch.sum(torch.stack(v), axis = 0) / len(v)).type(v[0].dtype) for k, v in soups.items() if len(v) != 0}
        model_dict.update(soups)
        model.load_state_dict(model_dict)
    return model


device = "cpu"
config_path = "/home/jovyan/work/work_space/uijin/archive/swin_k1/swin_L_K1.py"

################ soup할 checkpoint path 적기 ################
checkpoint_paths = [
    "/home/jovyan/work/work_space/uijin/archive/swin_k1/best_mDice_iter_200000.pth",
    "/home/jovyan/work/work_space/uijin/archive/swin_k2/best_mDice_iter_240000.pth",
    "/home/jovyan/work/work_space/uijin/archive/swin_k3/best_mDice_iter_240000.pth",
    "/home/jovyan/work/work_space/uijin/archive/swin_k4/best_mDice_iter_192000.pth",]

################ save dir path 적기 ################
save_dir_path = ""
name = 'mysoup' # soup 이름 적기
################ save dir path 적기 ################

print("\n[Uniform Soup]")
uniform_model = uniform_soup(config_path, checkpoint_paths, device = device)
uniform_dict['state_dict'] = uniform_model.state_dict()
torch.save(uniform_model.state_dict(), f'./uniform_model_soup_{name}.pth')
print("complete!")


[Uniform Soup]
['/home/jovyan/work/work_space/uijin/archive/swin_k1/best_mDice_iter_200000.pth', '/home/jovyan/work/work_space/uijin/archive/swin_k2/best_mDice_iter_240000.pth', '/home/jovyan/work/work_space/uijin/archive/swin_k3/best_mDice_iter_240000.pth', '/home/jovyan/work/work_space/uijin/archive/swin_k4/best_mDice_iter_192000.pth']
Loads checkpoint by local backend from path: /home/jovyan/work/work_space/uijin/archive/swin_k1/best_mDice_iter_200000.pth
Loads checkpoint by local backend from path: /home/jovyan/work/work_space/uijin/archive/swin_k1/best_mDice_iter_200000.pth
Loads checkpoint by local backend from path: /home/jovyan/work/work_space/uijin/archive/swin_k2/best_mDice_iter_240000.pth
Loads checkpoint by local backend from path: /home/jovyan/work/work_space/uijin/archive/swin_k3/best_mDice_iter_240000.pth
Loads checkpoint by local backend from path: /home/jovyan/work/work_space/uijin/archive/swin_k4/best_mDice_iter_192000.pth
complete!
